# 3. Sharpness-aware minimization (SAM) with Sparse Networks - 25 points

## 3.1 Get a sparse networks through pruning
**Pruning** is a technique used to reduce the size and complexity of a neural network model by removing (setting to zero) less important parameters. The goal is to create a more efficient model that retains its predictive accuracy while being smaller, which can improve both inference speed and memory usage.

Let's train a simple model on the MNIST dataset to learn about pruning at first. We just use 10% of the dataset for both training and testing.

### 3.1.1 Train a dense network with SGD
Let us first train a dense model with SGD. We reuse the model for the discriminator of the GAN in Homework 2 and name it 'Classifier'.

In [1]:
from lib.part3.utils import *
max_epochs = 10
device = "cpu" # Change this if you can and want to use a GPU device
model = Classifier().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

We define the optimizing process of SGD.

In [2]:
def optimize_sgd(model, optimizer, img, label):
    optimizer.zero_grad()
    output = model(img)
    loss = cross_entropy(output, label)
    loss.backward()
    optimizer.step()

The following cell runs the training loop, this might take a few minutes.

In [3]:
train_model(model, optimizer, optimize_sgd, max_epochs)

Epoch 0 with 0.931 accuracy on the validation set.
Epoch 1 with 0.946 accuracy on the validation set.
Epoch 2 with 0.954 accuracy on the validation set.
Epoch 3 with 0.957 accuracy on the validation set.
Epoch 4 with 0.963 accuracy on the validation set.
Epoch 5 with 0.966 accuracy on the validation set.
Epoch 6 with 0.967 accuracy on the validation set.
Epoch 7 with 0.971 accuracy on the validation set.
Epoch 8 with 0.971 accuracy on the validation set.
Epoch 9 with 0.973 accuracy on the validation set.


#### Evaluate model
Evaluate the model on the test set.

In [4]:
acc = evaluate(model)
print(f"Accuracy of {round(acc, 4)} on the test set.")

Accuracy of 0.9771 on the test set.


### 3.1.2 Sparse network with magnitude-based pruning

Magnitude-based pruning specifically focuses on **removing weights that have the smallest absolute values**, under the assumption that weights with smaller magnitudes contribute less to the model's output.

**(1)** (6 points) Realize magnitude-based pruning below, which removes a part of weights that have the smallest absolute values.

In [5]:
def magnitude_prune(model, prune_fraction):
    for name, param in model.named_parameters():
        if "weight" in name and param.requires_grad:
            # FILL: Get weight's absolute values
            weight_abs = abs(param.data)
            # FILL: Compute the threshold
            threshold = torch.quantile(weight_abs.flatten(), prune_fraction)
            # FILL: Prune weights below the threshold
            weight_abs[weight_abs < threshold] = 0
            weight_mask = (weight_abs != 0).float()
            param.data *= weight_mask
    return model

In [6]:
import copy
# Copy a model for pruning
sparse_model = copy.deepcopy(model)
# Get a sparse model by pruning 50% parameters
sparse_model = magnitude_prune(sparse_model, prune_fraction=0.5)

Copy a sparse model for SAM implementation later in 3.2.

In [7]:
sparse_model_sam = copy.deepcopy(sparse_model)

Evaluation after pruning

In [8]:
acc = evaluate(sparse_model)
print(f"Accuracy of {round(acc, 4)} on the test set.")

Accuracy of 0.948 on the test set.


### 3.1.3 Finetune the sparse model

We finetune the sparse model after pruning with SGD to recover its performance.

In [9]:
finetune_epoch = 3
train_model(sparse_model, optimizer, optimize_sgd, finetune_epoch)

Epoch 0 with 0.942 accuracy on the validation set.
Epoch 1 with 0.942 accuracy on the validation set.
Epoch 2 with 0.942 accuracy on the validation set.


Evaluate the sparse model after finetuning.

In [10]:
acc = evaluate(sparse_model)
print(f"Accuracy of {round(acc, 4)} on the test set.")

Accuracy of 0.9544 on the test set.


**(2)** (2 point) What are the pros and cons of sparse networks?

<font color=‘blue’>

**Answer:** 
The pros of sparse networks are that they reduce the number of parameters by setting them to zero, which leads to lower memory usage and faster inference speed. Pruning also helps models to learn more robust features, which improves generalization and reduces overfitting. However, the prune fraction (used to determine the threshold) must be carefully chosen to avoid losing important information for the model. 

The cons are that the accuracy or performance of these models might be worse than models that do not use pruning. Moreover, the training process can be harder to optimize and may show slower convergence. In our above implementation, after pruning the model, the accuracy dropped from 97% to 95%, and despite trying different numbers of epochs for finetuning, the accuracy got stuck during the training.

## 3.2 Train the sparse model with SAM

Sharpness-aware minimization (SAM) is a new optimization technique, which is satisfied with not just a low loss, instead it seeks a neighborhood with uniformly low loss. SAM is motivated by the link between the geometry of the loss landscape and generalization. It makes sense that a low loss within a uniformly low loss neighborhood will generalize better than a low loss within a region of higher variance.

To be specific, we consider a model with the weight vector of $\mathbf{w}$ and the training loss $L_S$. SAM aims to minimize the maximum loss within a small region which is usually a $\ell_2$ ball with $\rho$ radius. Note that $\rho$ is a small value close to $0$. Therefore, SAM can be formulated as a minimax optimization problem:
$$\min_{\mathbf{w}} \max_{\mathbf{\epsilon}: \|\mathbf{\epsilon}\|_2\leq \rho} L_S (\mathbf{w} + \mathbf{\epsilon})$$

**(3)** (3 points) Please solve the inner maximum problem by first-order Taylor expansion.

<font color=‘blue’>

**Solution:**

First-order Taylor series (linear approximation):
$$
f(x) \approx f(x_0) + f'(x_0)(x-x_0)
$$

Inner maximum problem: 
$$
\max_{\varepsilon: \|\varepsilon\|_2 \leq \rho} L_s(w + \varepsilon) \text{ where } \rho \approx 0
$$

1° Applying first-order series considering the following:
$$
f(x) = L_s(w+\varepsilon), \quad f'(x_0) = \nabla_w L_s(w), \quad f(x_0) = L_s(w)
$$

$$
L_s(w+\varepsilon) \approx L_s(w) + \nabla_w L_s(w)^T(w+\varepsilon-w)
$$
$$
L_s(w+\varepsilon) \approx L_s(w) + \nabla_w L_s(w)^T\varepsilon
$$

2° Then the maximization problem can be written as:
$$
\max_{\varepsilon: \|\varepsilon\|_2 \leq \rho} L_s(w) + \nabla_w L_s(w)^T\varepsilon
$$

3° We must only focus on $\nabla_w L_s(w)^T\varepsilon$ since it depends on $\varepsilon$:
$$
\max_{\varepsilon: \|\varepsilon\|_2 \leq \rho} \nabla_w L_s(w)^T\varepsilon
$$

4° Now, we must find $\varepsilon^*$ that maximizes the dot product:
$$
\nabla_w L_s(w)^T\varepsilon^* = \|\nabla_w L_s(w)\|_2 \|\varepsilon^*\|_2 \cos(\theta)
$$

5° When these two vectors points to the same direction $\theta = 0$, we get the maximum value. Thus, the $\varepsilon^*$ should be aligned in the direction of $\nabla_w L_s(w)$:
$$
\varepsilon^* = k\nabla_w L_s(w)
$$

6° $k$ is a scalar multiple of $\nabla_w L_s(w)$. However, we must be careful with the constraint: $\varepsilon: \|\varepsilon\|_2 \leq \rho$. Therefore we can get the unit vector of $\nabla_w L_s(w)$ to ensure the direction is correct:
$$
\varepsilon^* = k\frac{\nabla_w L_s(w)}{\|\nabla_w L_s(w)\|_2}
$$

7° Finally, $k$ can take the maximum value of $\|\varepsilon\|_2$ that is $\rho$, because multiplying the unit vector with $\rho$ satisfies the constraint, which gives us the optimal perturbation $\varepsilon^*$:
$$
\varepsilon^* = \rho\frac{\nabla_w L_s(w)}{\|\nabla_w L_s(w)\|_2}
$$

**(4)** (8 points) Now we will train the same model using the SAM optimizer.
Please implement SAM by the two steps below. The first step is for the maximizer which calculates $\epsilon$ obtained in question (1). The second step is the normal step for the minimizer: $\mathbf{w}_{t+1} = \mathbf{w}_{t} - \eta_t \nabla L_S (\mathbf{w}_t + \mathbf{\epsilon}_t)$ where $\eta_t$ is step size. Note that we set $\rho=0.05$.

Hint: be careful about weight updates.

In [11]:
class SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, lr=0.01, rho=0.05):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho)
        super(SAM, self).__init__(params, defaults)

        self.base_optimizer = base_optimizer(self.param_groups, lr)
        self.param_groups = self.base_optimizer.param_groups
        self.defaults.update(self.base_optimizer.defaults)

    def _grad_norm(self):
        # Note that p.grad gets the gradient; p.data gets the weight.
        norm = torch.norm(
                    torch.stack([
                        p.grad.norm(p=2)
                        for group in self.param_groups for p in group["params"]
                        if p.grad is not None
                    ]),
                    p=2
               )
        norm += 1e-12 # Avoid zero norm
        return norm

    @torch.no_grad()
    def first_step(self):
        # Add the perturbation on the weight.
        # Hint: the norm of the gradient can be calculated by _grad_norm() function.
        # Hint: self.param_groups to get access to the weight and gradient of each parameter
        # FILL
        grad_norm = self._grad_norm()
        
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is not None:
                    epsilon = self.defaults['rho'] * p.grad / grad_norm
                    p.data += epsilon
                    self.state['perturbations'][p] = epsilon

        self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        # FILL
        # Hint: Remember to change the parameters back
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is not None:
                    p.data -= self.state['perturbations'][p]
        
        self.state['perturbations'] = {}
            
        self.base_optimizer.step()

        self.zero_grad()

Define an optimizer of `SAM` for the model. We recommend using `SGD` as base optimizer with a learning rate of $0.05$ (which is same with SGD).

In [12]:
base_optimizer = torch.optim.SGD
sam_optimizer = SAM(sparse_model_sam.parameters(), base_optimizer, lr=0.05)

**(5)** (4 points) Please define the optimizing process of SAM.

In [13]:
def optimize_sam(model, optimizer, img, label):
    enable_running_stats(model)
    # First forward-backward pass
    # FILL
    # Hint: use sam_optimizer above
    # Hint: use loss 'cross_entropy'
    pred = model(img)
    loss = cross_entropy(pred, label)
    loss.backward()
    optimizer.first_step()

    disable_running_stats(model)
    # Second forward-backward pass
    # FILL
    pred = model(img)
    loss = cross_entropy(pred, label)
    loss.backward()
    optimizer.second_step()

In [14]:
train_model(sparse_model_sam, sam_optimizer, optimize_sam, finetune_epoch)

Epoch 0 with 0.964 accuracy on the validation set.
Epoch 1 with 0.966 accuracy on the validation set.
Epoch 2 with 0.964 accuracy on the validation set.


#### Evaluate model
Evaluate the sparse model finetuned with SAM on the test set.

In [ ]:
acc = evaluate(sparse_model_sam)
print(f"Accuracy of {round(acc, 4)} on the test set.")

Accuracy of 0.9789 on the test set.


**(6)** (2 points) Give a conclusion comparing SAM with SGD. Is there any drawback of SAM?

<font color=‘blue’>
    
**Answer:**
SAM improves generalization by seeking parameters that are in neighborhoods with uniformly low loss. In contrast, SGD only focuses on minimizing the loss considering the current parameters. As a result of applying both methods in the previous problem, we can see that SAM achieved a higher accuracy of 97.8% compared to the 95% obtained by SGD on the test set. However, SGD is clearly faster than SAM in terms of training time. SAM requires two forward-backward passes for each iteration, which makes it slower than SGD. Moreover, SAM needs to store the perturbations to change the parameters back in the second optimization step, which increases memory usage. Therefore, the main drawbacks of SAM are the computational cost and memory usage, which can be a problem for larger models.